In [1]:
import pandas as pd
import numpy as np

In [4]:
#parameter tagged cell for use with shell script
year = 2015
quarter = 'QTR4'

In [5]:
nd= pd.read_csv(f'/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Grouped_Insiders/Buys_Only/{year}_{quarter}.csv' )

In [6]:
nd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8709 entries, 0 to 8708
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   transaction_date             8329 non-null   object 
 1   total_amount_bought_or_sold  8709 non-null   int64  
 2   total_dollar_bought_or_sold  8709 non-null   int64  
 3   purchase_price               8329 non-null   float64
 4   amount_owned_following       8709 non-null   float64
 5   url                          8709 non-null   object 
 6   reporting_date               8709 non-null   object 
 7   ticker                       8709 non-null   object 
 8   name                         8709 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 612.5+ KB


In [34]:
rfb= nd

In [35]:
import re

path_string = f"/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/Grouped_Insiders/Buys_Only/{year}_{quarter}.csv"
match = re.search(r'Buys_Only/([^/]+)\.csv', path_string)

if match:
    extracted_string = match.group(1)
    print(extracted_string)  
else:
    print("Pattern not found in the string")


2015_QTR4


In [36]:
extracted_string

'2015_QTR4'

In [37]:
#concat the results of the two date fields if one is missing, prioritizing reporting_date

from dateutil.parser import parse
import pandas as pd

# updated function to handle strings that are not in a date format
def clean_date(date_str):
    if isinstance(date_str, str):
        if '(' in date_str:
            date_str = date_str.split('(')[0].strip()
        try:
            # try parsing the date string to see if it's in a valid date format
            parse(date_str)
            return date_str
        except ValueError:
            # if it's not a valid date, return NaN
            return pd.NaT
    return date_str

# apply the cleaning function to your date columns in rfb
rfb['transaction_date'] = rfb['transaction_date'].apply(clean_date)
rfb['reporting_date'] = rfb['reporting_date'].apply(clean_date)

# convert the date columns from strings to datetime objects in rfb
rfb['transaction_date'] = pd.to_datetime(rfb['transaction_date'], errors='coerce')
rfb['reporting_date'] = pd.to_datetime(rfb['reporting_date'], errors='coerce')



In [38]:
rfb.reset_index(drop= True )

,transaction_date,total_amount_bought_or_sold,total_dollar_bought_or_sold,amount_owned_following,url,reporting_date,ticker,name
0,2015-11-23,1030103,10949994,3754828.0,https://sec.gov/Archives/edgar/data/1476548/0001144204-15-068119-index.html,2015-11-25,GDEF,Perl Andrew Damian
1,2015-12-09,12700,5753,1350000.0,https://sec.gov/Archives/edgar/data/1477009/0001387131-15-003625-index.html,2015-12-15,CANN,FEINSOD MICHAEL
2,2015-09-30,7872,46764,611986.0,https://sec.gov/Archives/edgar/data/1477025/0001062993-15-005270-index.html,2015-10-02,BVSN,"ESW Capital, LLC"
3,2015-09-30,4674,36503,2088131.0,https://sec.gov/Archives/edgar/data/1477025/0001062993-15-005271-index.html,2015-10-02,UPLD,"ESW Capital, LLC"
4,2015-10-13,371,2226,637089.0,https://sec.gov/Archives/edgar/data/1477025/0001062993-15-005421-index.html,2015-10-15,BVSN,"ESW Capital, LLC"
...,...,...,...,...,...,...,...,...
8704,2015-11-27,15000,85650,354808.0,https://sec.gov/Archives/edgar/data/1113465/0001113465-15-000008-index.html,2015-12-01,ACTG,HARRIS ROBERT L II
8705,2015-11-30,10500,59430,365308.0,https://sec.gov/Archives/edgar/data/1113465/0001113465-15-000009-index.html,2015-12-02,ACTG,HARRIS ROBERT L II
8706,2015-09-30,81515,32883,9009783.0,https://sec.gov/Archives/edgar/data/1113835/0001493152-15-004671-index.html,2015-10-02,WIZD,KESSLER PAUL
8707,2015-12-01,3000000,9900,40613656.0,https://sec.gov/Archives/edgar/data/1114208/0001477932-15-007440-index.html,2015-12-01,NGHT,Surain Adyanthaya


In [39]:
rfb = rfb.drop_duplicates(subset=['ticker', 'reporting_date', 'total_amount_bought_or_sold', 'name'])

In [41]:
# replace null values of transaction_date with reporting_date
rfb['transaction_date'] = rfb['transaction_date'].fillna(rfb['reporting_date'])


/var/folders/v5/xxtsm0vd1hx9g9n9w41gcrf80000gn/T/ipykernel_28871/2471129097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfb['transaction_date'] = rfb['transaction_date'].fillna(rfb['reporting_date'])


In [42]:
rfb.dropna(inplace=True)

/Users/rileybitterli/SB/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [43]:
rfb.reset_index(drop=True, inplace=True)

In [44]:
rfb

,transaction_date,total_amount_bought_or_sold,total_dollar_bought_or_sold,amount_owned_following,url,reporting_date,ticker,name
0,2015-11-23,1030103,10949994,3754828.0,https://sec.gov/Archives/edgar/data/1476548/0001144204-15-068119-index.html,2015-11-25,GDEF,Perl Andrew Damian
1,2015-12-09,12700,5753,1350000.0,https://sec.gov/Archives/edgar/data/1477009/0001387131-15-003625-index.html,2015-12-15,CANN,FEINSOD MICHAEL
2,2015-09-30,7872,46764,611986.0,https://sec.gov/Archives/edgar/data/1477025/0001062993-15-005270-index.html,2015-10-02,BVSN,"ESW Capital, LLC"
3,2015-09-30,4674,36503,2088131.0,https://sec.gov/Archives/edgar/data/1477025/0001062993-15-005271-index.html,2015-10-02,UPLD,"ESW Capital, LLC"
4,2015-10-13,371,2226,637089.0,https://sec.gov/Archives/edgar/data/1477025/0001062993-15-005421-index.html,2015-10-15,BVSN,"ESW Capital, LLC"
...,...,...,...,...,...,...,...,...
4585,2015-11-09,40000,94471,436224.0,https://sec.gov/Archives/edgar/data/1113232/0001209191-15-079201-index.html,2015-11-10,ACLS,PUMA MARY G
4586,2015-11-30,10500,59430,365308.0,https://sec.gov/Archives/edgar/data/1113465/0001113465-15-000009-index.html,2015-12-02,ACTG,HARRIS ROBERT L II
4587,2015-09-30,81515,32883,9009783.0,https://sec.gov/Archives/edgar/data/1113835/0001493152-15-004671-index.html,2015-10-02,WIZD,KESSLER PAUL
4588,2015-12-01,3000000,9900,40613656.0,https://sec.gov/Archives/edgar/data/1114208/0001477932-15-007440-index.html,2015-12-01,NGHT,Surain Adyanthaya


In [22]:
#enrich with YQ data

In [45]:
import yahooquery as yq
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay as BDay
from dateutil import parser

In [46]:
rfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4590 entries, 0 to 4589
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   transaction_date             4590 non-null   datetime64[ns]
 1   total_amount_bought_or_sold  4590 non-null   int64         
 2   total_dollar_bought_or_sold  4590 non-null   int64         
 3   amount_owned_following       4590 non-null   float64       
 4   url                          4590 non-null   object        
 5   reporting_date               4590 non-null   datetime64[ns]
 6   ticker                       4590 non-null   object        
 7   name                         4590 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 287.0+ KB


In [47]:
reporting_dates = rfb['reporting_date']

In [48]:
reporting_dates

0      2015-11-25
1      2015-12-15
2      2015-10-02
3      2015-10-02
4      2015-10-15
          ...    
4585   2015-11-10
4586   2015-12-02
4587   2015-10-02
4588   2015-12-01
4589   2015-11-05
Name: reporting_date, Length: 4590, dtype: datetime64[ns]

In [49]:
forty_five = [reporting_date + BDay(22) for reporting_date in reporting_dates]

In [50]:
minus_30 =  [reporting_date - BDay(22) for reporting_date in reporting_dates]

In [51]:
minus_10 =  [reporting_date - BDay(7) for reporting_date in reporting_dates]

In [119]:
#somehow the BDay function doesn't know new years day isn't one. sad.

In [52]:
dates = list(zip(list(rfb['ticker']), reporting_dates, forty_five, minus_10, minus_30))

In [1]:
len(dates)

NameError: name 'dates' is not defined

In [54]:
import concurrent.futures
import yahooquery as yq
import numpy as np

def fetch_data_for_date(lol):
    # initialize result dictionary with all required fields
    results = {
        'vix_on_dof': None,
        'tnx_on_dof': None,
        'dof_price': None,
        'lookahead_price': None,
        'forty_five_avg': None,
        'minus_10_price': None, 
        'minus_30_price': None  
    }
    try:
        # fetch VIX and TNX data on the date of interest
        try:
            results['vix_on_dof'] = yq.Ticker('^VIX').history(start=lol[1]-BDay(2), end=lol[1])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['vix_on_dof'] = np.nan

        try:
            results['tnx_on_dof'] = yq.Ticker('^TNX').history(start=lol[1]-BDay(2), end=lol[1])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['tnx_on_dof'] = np.nan

        # fetch stock prices for the main date and lookahead date
        holder = []
        try:
            holder = yq.Ticker(lol[0]).history(start=lol[1], end=lol[2])['close']
        except (IndexError, KeyError, AttributeError):
            pass

        try:
            results['dof_price'] = np.mean(holder[:3])
        except (IndexError, KeyError, AttributeError):
            results['dof_price'] = np.nan

        try:
            results['lookahead_price'] = np.mean(holder[-3:])
        except (IndexError, KeyError, AttributeError):
            results['lookahead_price'] = np.nan

        # calculate 45-day average
        try:
            results['forty_five_avg'] = holder.mean()
        except (IndexError, KeyError, AttributeError):
            results['forty_five_avg'] = np.nan

        # fetch stock price for minus_10
        try:
            results['minus_10_price'] = yq.Ticker(lol[0]).history(start=lol[3]-BDay(2), end=lol[3])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['minus_10_price'] = np.nan

        # fetch stock price for minus_30
        try:
            results['minus_30_price'] = yq.Ticker(lol[0]).history(start=lol[4]-BDay(2), end=lol[4])['close'].mean()
        except (IndexError, KeyError, AttributeError):
            results['minus_30_price'] = np.nan
    
    except (IndexError, KeyError, AttributeError, TypeError) as e:
        results['vix_on_dof'] = np.nan
        results['tnx_on_dof'] = np.nan
        results['dof_price'] = np.nan
        results['lookahead_price'] = np.nan
        results['forty_five_avg'] = np.nan
        results['minus_10_price'] = np.nan
        results['minus_30_price'] = np.nan
            
    return results

# lists to store results
dof_price = []
lookahead_price = []
forty_five_avg = []
vix_on_dof = []
tnx_on_dof = []
minus_10_price = [] 
minus_30_price = []  


with concurrent.futures.ThreadPoolExecutor() as executor:
    for index, result in enumerate(executor.map(fetch_data_for_date, dates)):
        dof_price.append(result['dof_price'])
        lookahead_price.append(result['lookahead_price'])
        forty_five_avg.append(result['forty_five_avg'])
        vix_on_dof.append(result['vix_on_dof'])
        tnx_on_dof.append(result['tnx_on_dof'])
        minus_10_price.append(result['minus_10_price'])
        minus_30_price.append(result['minus_30_price'])
        print("Processed index:", index + 1)

print('all done')


Processed index: 1
Processed index: 2
Processed index: 3
Processed index: 4
Processed index: 5
Processed index: 6
Processed index: 7
Processed index: 8
Processed index: 9
Processed index: 10
Processed index: 11
Processed index: 12
Processed index: 13
Processed index: 14
Processed index: 15
Processed index: 16
Processed index: 17
Processed index: 18
Processed index: 19
Processed index: 20
Processed index: 21
Processed index: 22
Processed index: 23
Processed index: 24
Processed index: 25
Processed index: 26
Processed index: 27
Processed index: 28
Processed index: 29
Processed index: 30
Processed index: 31
Processed index: 32
Processed index: 33
Processed index: 34
Processed index: 35
Processed index: 36
Processed index: 37
Processed index: 38
Processed index: 39
Processed index: 40
Processed index: 41
Processed index: 42
Processed index: 43
Processed index: 44
Processed index: 45
Processed index: 46
Processed index: 47
Processed index: 48
Processed index: 49
Processed index: 50
Processed

Processed index: 397
Processed index: 398
Processed index: 399
Processed index: 400
Processed index: 401
Processed index: 402
Processed index: 403
Processed index: 404
Processed index: 405
Processed index: 406
Processed index: 407
Processed index: 408
Processed index: 409
Processed index: 410
Processed index: 411
Processed index: 412
Processed index: 413
Processed index: 414
Processed index: 415
Processed index: 416
Processed index: 417
Processed index: 418
Processed index: 419
Processed index: 420
Processed index: 421
Processed index: 422
Processed index: 423
Processed index: 424
Processed index: 425
Processed index: 426
Processed index: 427
Processed index: 428
Processed index: 429
Processed index: 430
Processed index: 431
Processed index: 432
Processed index: 433
Processed index: 434
Processed index: 435
Processed index: 436
Processed index: 437
Processed index: 438
Processed index: 439
Processed index: 440
Processed index: 441
Processed index: 442
Processed index: 443
Processed ind

Processed index: 793
Processed index: 794
Processed index: 795
Processed index: 796
Processed index: 797
Processed index: 798
Processed index: 799
Processed index: 800
Processed index: 801
Processed index: 802
Processed index: 803
Processed index: 804
Processed index: 805
Processed index: 806
Processed index: 807
Processed index: 808
Processed index: 809
Processed index: 810
Processed index: 811
Processed index: 812
Processed index: 813
Processed index: 814
Processed index: 815
Processed index: 816
Processed index: 817
Processed index: 818
Processed index: 819
Processed index: 820
Processed index: 821
Processed index: 822
Processed index: 823
Processed index: 824
Processed index: 825
Processed index: 826
Processed index: 827
Processed index: 828
Processed index: 829
Processed index: 830
Processed index: 831
Processed index: 832
Processed index: 833
Processed index: 834
Processed index: 835
Processed index: 836
Processed index: 837
Processed index: 838
Processed index: 839
Processed ind

Processed index: 1178
Processed index: 1179
Processed index: 1180
Processed index: 1181
Processed index: 1182
Processed index: 1183
Processed index: 1184
Processed index: 1185
Processed index: 1186
Processed index: 1187
Processed index: 1188
Processed index: 1189
Processed index: 1190
Processed index: 1191
Processed index: 1192
Processed index: 1193
Processed index: 1194
Processed index: 1195
Processed index: 1196
Processed index: 1197
Processed index: 1198
Processed index: 1199
Processed index: 1200
Processed index: 1201
Processed index: 1202
Processed index: 1203
Processed index: 1204
Processed index: 1205
Processed index: 1206
Processed index: 1207
Processed index: 1208
Processed index: 1209
Processed index: 1210
Processed index: 1211
Processed index: 1212
Processed index: 1213
Processed index: 1214
Processed index: 1215
Processed index: 1216
Processed index: 1217
Processed index: 1218
Processed index: 1219
Processed index: 1220
Processed index: 1221
Processed index: 1222
Processed 

TypeError: an integer is required (got type NoneType)

In [ ]:
rfb['dof_price'] = dof_price
rfb['30_day_price'] = lookahead_price
rfb['avg_30_price'] = forty_five_avg
rfb['vix_on_dof'] = vix_on_dof
rfb['tnx_on_dof'] = tnx_on_dof
rfb['minus_10_price'] = minus_10_price
rfb['minus_30_price'] = minus_30_price

In [77]:
#begin proper feature engineering

In [79]:
# accounting for very small edge case where the company hasn't been around for 30 days yet. 
import numpy as np

# for rows where minus_30_price is NaN and minus_10_price is not NaN, replace minus_30_price with minus_10_price
rfb['minus_30_price'] = np.where(rfb['minus_30_price'].isna() & rfb['minus_10_price'].notna(), 
                                 rfb['minus_10_price'], 
                                 rfb['minus_30_price'])

# for rows where both minus_30_price and minus_10_price are NaN but dof_price isn't, 
# replace both minus_30_price and minus_10_price with dof_price
condition = rfb['minus_30_price'].isna() & rfb['minus_10_price'].isna() & rfb['dof_price'].notna()
rfb.loc[condition, 'minus_30_price'] = rfb['dof_price']
rfb.loc[condition, 'minus_10_price'] = rfb['dof_price']


In [80]:
rfb['ticker'] = rfb['ticker'].str.strip()

In [81]:
rfb['shares_percent_increased'] = rfb['total_amount_bought_or_sold']/ (abs(rfb['total_amount_bought_or_sold']) + rfb['amount_owned_following'])

In [82]:
rfb = rfb.drop_duplicates()

In [83]:
rfb.dropna(inplace=True)

In [84]:
rfb.reset_index(inplace = True, drop= True)

In [85]:
rfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   transaction_date             2208 non-null   datetime64[ns]
 1   total_amount_bought_or_sold  2208 non-null   int64         
 2   total_dollar_bought_or_sold  2208 non-null   int64         
 3   amount_owned_following       2208 non-null   float64       
 4   url                          2208 non-null   object        
 5   reporting_date               2208 non-null   datetime64[ns]
 6   ticker                       2208 non-null   object        
 7   name                         2208 non-null   object        
 8   dof_price                    2208 non-null   float64       
 9   30_day_price                 2208 non-null   float64       
 10  avg_30_price                 2208 non-null   float64       
 11  vix_on_dof                   2208 non-null 

In [86]:
rfb['10_day_delta'] = (rfb['dof_price']- rfb['minus_10_price']) / rfb['minus_10_price']
rfb['30_day_delta'] = (rfb['dof_price']- rfb['minus_30_price']) / rfb['minus_30_price']

In [87]:
rfb['dof_vs_30_avg'] = (rfb['avg_30_price'] - rfb['dof_price']) / rfb['dof_price']
rfb['dof_vs_30_actual'] = (rfb['30_day_price'] - rfb['dof_price']) / rfb['dof_price']

In [ ]:
rfb['purchase_price_vs_price_on_dof'] = ((rfb['purchase_price'] - rfb['dof_price']) / rfb['dof_price'])*100

In [88]:
engineered_df = rfb[['ticker',
                     'reporting_date',
                     'url',
                     'total_amount_bought_or_sold',
                     'total_dollar_bought_or_sold',
                     'amount_owned_following',
                     'purchase_price',
                     'shares_percent_increased',
                     'dof_price',
                     'vix_on_dof',
                     'tnx_on_dof',
                     'minus_10_price',
                     '10_day_delta',
                     'purchase_price_vs_price_on_dof',
                     '30_day_delta',
                     'minus_30_price',
                      'dof_vs_30_avg',
                     'dof_vs_30_actual'
                     
                     
                     
                     
                     
    
]]

In [89]:
engineered_df

,ticker,reporting_date,url,total_amount_bought_or_sold,total_dollar_bought_or_sold,amount_owned_following,shares_percent_increased,dof_price,vix_on_dof,tnx_on_dof,minus_10_price,10_day_delta,30_day_delta,minus_30_price,dof_vs_30_avg,dof_vs_30_actual
0,PDCO,2017-11-27,https://sec.gov/Archives/edgar/data/1289177/0001209191-17-062555-index.html,2834,98991,54627.000,0.049320,36.446668,9.770000,2.3400,33.990000,0.072276,-0.084370,39.805000,-0.008636,-0.004939
1,NATR,2017-11-21,https://sec.gov/Archives/edgar/data/1289453/0001144204-17-060426-index.html,8739,102245,1508617.000,0.005759,12.450000,10.603333,2.3630,10.575000,0.177305,0.223587,10.175000,-0.011092,-0.016064
2,CINF,2017-12-05,https://sec.gov/Archives/edgar/data/1289454/0001127602-17-033892-index.html,134,9999,26740.379,0.004986,74.333333,11.480000,2.3705,72.660004,0.023030,0.037668,71.634998,-0.003771,-0.009686
3,NURO,2017-10-23,https://sec.gov/Archives/edgar/data/1289850/0001628280-17-010032-index.html,2500,4419,18463.000,0.119258,137.866664,10.363333,2.3510,166.000000,-0.169478,-0.131822,158.800003,0.040267,0.081238
4,NURO,2017-10-24,https://sec.gov/Archives/edgar/data/1289850/0001628280-17-010102-index.html,2500,4330,29863.000,0.077249,139.199997,10.733333,2.3785,176.000000,-0.209091,-0.138614,161.599998,0.033438,0.063218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,NTWK,2017-12-06,https://sec.gov/Archives/edgar/data/1258670/0001493152-17-014233-index.html,1000,4429,61850.000,0.015911,4.366667,11.343334,2.3675,4.200000,0.039683,0.303483,3.350000,0.065573,0.091603
2204,CTHR,2017-11-13,https://sec.gov/Archives/edgar/data/1258861/0001144204-17-058473-index.html,7614,8288,795700.000,0.009478,1.056667,11.096667,2.3655,0.855000,0.235867,0.200758,0.880000,0.177107,0.353312
2205,CTHR,2017-11-15,https://sec.gov/Archives/edgar/data/1258861/0001144204-17-059500-index.html,12799,13510,808499.000,0.015584,1.080000,12.073333,2.3905,0.890000,0.213483,0.248555,0.865000,0.182540,0.287037
2206,CTHR,2017-11-29,https://sec.gov/Archives/edgar/data/1258861/0001144204-17-061553-index.html,24148,31342,832647.000,0.028184,1.296667,10.200000,2.3330,1.090000,0.189602,0.516569,0.855000,0.055821,0.048843


In [90]:
final = engineered_df

In [91]:
final = final.dropna()

In [92]:
final.reset_index(inplace=True, drop=True)

In [93]:
final

,ticker,reporting_date,url,total_amount_bought_or_sold,total_dollar_bought_or_sold,amount_owned_following,shares_percent_increased,dof_price,vix_on_dof,tnx_on_dof,minus_10_price,10_day_delta,30_day_delta,minus_30_price,dof_vs_30_avg,dof_vs_30_actual
0,PDCO,2017-11-27,https://sec.gov/Archives/edgar/data/1289177/0001209191-17-062555-index.html,2834,98991,54627.000,0.049320,36.446668,9.770000,2.3400,33.990000,0.072276,-0.084370,39.805000,-0.008636,-0.004939
1,NATR,2017-11-21,https://sec.gov/Archives/edgar/data/1289453/0001144204-17-060426-index.html,8739,102245,1508617.000,0.005759,12.450000,10.603333,2.3630,10.575000,0.177305,0.223587,10.175000,-0.011092,-0.016064
2,CINF,2017-12-05,https://sec.gov/Archives/edgar/data/1289454/0001127602-17-033892-index.html,134,9999,26740.379,0.004986,74.333333,11.480000,2.3705,72.660004,0.023030,0.037668,71.634998,-0.003771,-0.009686
3,NURO,2017-10-23,https://sec.gov/Archives/edgar/data/1289850/0001628280-17-010032-index.html,2500,4419,18463.000,0.119258,137.866664,10.363333,2.3510,166.000000,-0.169478,-0.131822,158.800003,0.040267,0.081238
4,NURO,2017-10-24,https://sec.gov/Archives/edgar/data/1289850/0001628280-17-010102-index.html,2500,4330,29863.000,0.077249,139.199997,10.733333,2.3785,176.000000,-0.209091,-0.138614,161.599998,0.033438,0.063218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,NTWK,2017-12-06,https://sec.gov/Archives/edgar/data/1258670/0001493152-17-014233-index.html,1000,4429,61850.000,0.015911,4.366667,11.343334,2.3675,4.200000,0.039683,0.303483,3.350000,0.065573,0.091603
2204,CTHR,2017-11-13,https://sec.gov/Archives/edgar/data/1258861/0001144204-17-058473-index.html,7614,8288,795700.000,0.009478,1.056667,11.096667,2.3655,0.855000,0.235867,0.200758,0.880000,0.177107,0.353312
2205,CTHR,2017-11-15,https://sec.gov/Archives/edgar/data/1258861/0001144204-17-059500-index.html,12799,13510,808499.000,0.015584,1.080000,12.073333,2.3905,0.890000,0.213483,0.248555,0.865000,0.182540,0.287037
2206,CTHR,2017-11-29,https://sec.gov/Archives/edgar/data/1258861/0001144204-17-061553-index.html,24148,31342,832647.000,0.028184,1.296667,10.200000,2.3330,1.090000,0.189602,0.516569,0.855000,0.055821,0.048843


In [94]:
final.to_csv(path_or_buf=f"/Users/rileybitterli/Desktop/StockBot_Data/Draft_VIII/YQ_Enriched/Buys/{extracted_string}.csv")

In [95]:
import smtplib
from email.message import EmailMessage
def send_email(subject, content):
    msg = EmailMessage()
    msg.set_content(content)
    msg['Subject'] = subject
    msg['From'] = 'halpitsstockbot@gmail.com'
    msg['To'] = 'riley.bitterli@gmail.com'

    # Establish a connection to Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login([insert your email address], [insert your password])
    server.send_message(msg)
    server.quit()

send_email('step 3', 'done')